In [1]:
import functools
from functools import reduce  # For Python 3.x

from pyspark.sql import DataFrame
from pyspark.sql import SparkSession

import pandas as pd
import time

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('vra_concat_sql')
    .getOrCreate()
)

In [31]:
df1 = spark.read.json('VRA_20211.json')
df2 = spark.read.json('VRA_20212.json')
df3 = spark.read.json('VRA_20213.json')
df4 = spark.read.json('VRA_20214.json')
df5 = spark.read.json('VRA_20215.json')
df6 = spark.read.json('VRA_20216.json')
df7 = spark.read.json('VRA_20217.json')
df8 = spark.read.json('VRA_20218.json')
df9 = spark.read.json('VRA_20219.json')
df10 = spark.read.json('VRA_202110.json')
df11 = spark.read.json('VRA_202111.json')



In [9]:
#def unionAll(dfs):
    #return functools.reduce(lambda df1, df: df1.union(df.select(df1.columns)), dfs)

In [32]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)
  
df_union = unionAll(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11)

In [33]:
df_union.show()

+-------------------+-------------------+-----------------+-------------------+---------------+--------------------+-------------------+----------------+---------+-------------------+-------------------+-----------+
|    ChegadaPrevista|        ChegadaReal|CódigoAutorização|CódigoJustificativa|CódigoTipoLinha|ICAOAeródromoDestino|ICAOAeródromoOrigem|ICAOEmpresaAérea|NúmeroVoo|    PartidaPrevista|        PartidaReal|SituaçãoVoo|
+-------------------+-------------------+-----------------+-------------------+---------------+--------------------+-------------------+----------------+---------+-------------------+-------------------+-----------+
|2021-01-19 14:45:00|2021-01-19 19:22:00|                7|                N/A|              C|                SBEG|               SBBE|             MWM|     9702|2021-01-19 12:35:00|2021-01-19 17:17:00|  REALIZADO|
|2021-01-26 12:05:00|2021-01-26 12:04:00|                7|                N/A|              C|                SBPA|               SBGL|

In [16]:
print('CódigoAutorização in DataFrame df is :', df_union.distinct().count())

CódigoAutorização in DataFrame df is : 535803


In [34]:
df_union.printSchema()

root
 |-- ChegadaPrevista: string (nullable = true)
 |-- ChegadaReal: string (nullable = true)
 |-- CódigoAutorização: string (nullable = true)
 |-- CódigoJustificativa: string (nullable = true)
 |-- CódigoTipoLinha: string (nullable = true)
 |-- ICAOAeródromoDestino: string (nullable = true)
 |-- ICAOAeródromoOrigem: string (nullable = true)
 |-- ICAOEmpresaAérea: string (nullable = true)
 |-- NúmeroVoo: string (nullable = true)
 |-- PartidaPrevista: string (nullable = true)
 |-- PartidaReal: string (nullable = true)
 |-- SituaçãoVoo: string (nullable = true)



In [35]:
df_union = df_union.withColumnRenamed('ChegadaPrevista', 'chegada_prevista')\
.withColumnRenamed('ChegadaReal', 'chegada_real')\
.withColumnRenamed('CódigoAutorização', 'código_autorização')\
.withColumnRenamed('CódigoJustificativa', 'código_justificativa')\
.withColumnRenamed('CódigoTipoLinha', 'código_tipoLinha')\
.withColumnRenamed('ICAOAeródromoDestino', 'icao_aeródromo_destino')\
.withColumnRenamed('ICAOAeródromoOrigem', 'icao_aeródromo_origem')\
.withColumnRenamed('ICAOEmpresaAérea', 'icao_empresa_aérea')\
.withColumnRenamed('NúmeroVoo', 'número_voo')\
.withColumnRenamed('PartidaPrevista', 'partida_prevista')\
.withColumnRenamed('PartidaReal', 'partida_real')\
.withColumnRenamed('SituaçãoVoo', 'situação_voo')\

df_union.show(5)

+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|código_autorização|código_justificativa|código_tipoLinha|icao_aeródromo_destino|icao_aeródromo_origem|icao_empresa_aérea|número_voo|   partida_prevista|       partida_real|situação_voo|
+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-01-19 14:45:00|2021-01-19 19:22:00|                 7|                 N/A|               C|                  SBEG|                 SBBE|               MWM|      9702|2021-01-19 12:35:00|2021-01-19 17:17:00|   REALIZADO|
|2021-01-26 12:05:00|2021-01-26 12:04:00|                 7|                 N/A|           

In [36]:
df_union.printSchema()

root
 |-- chegada_prevista: string (nullable = true)
 |-- chegada_real: string (nullable = true)
 |-- código_autorização: string (nullable = true)
 |-- código_justificativa: string (nullable = true)
 |-- código_tipoLinha: string (nullable = true)
 |-- icao_aeródromo_destino: string (nullable = true)
 |-- icao_aeródromo_origem: string (nullable = true)
 |-- icao_empresa_aérea: string (nullable = true)
 |-- número_voo: string (nullable = true)
 |-- partida_prevista: string (nullable = true)
 |-- partida_real: string (nullable = true)
 |-- situação_voo: string (nullable = true)



In [37]:
df_union.columns

['chegada_prevista',
 'chegada_real',
 'código_autorização',
 'código_justificativa',
 'código_tipoLinha',
 'icao_aeródromo_destino',
 'icao_aeródromo_origem',
 'icao_empresa_aérea',
 'número_voo',
 'partida_prevista',
 'partida_real',
 'situação_voo']

In [38]:
for coluna in df_union.columns:
    print(coluna, df_union.filter(df_union[coluna].isNull()).count())

chegada_prevista 1093
chegada_real 24245
código_autorização 0
código_justificativa 0
código_tipoLinha 0
icao_aeródromo_destino 0
icao_aeródromo_origem 0
icao_empresa_aérea 0
número_voo 0
partida_prevista 1093
partida_real 24245
situação_voo 0


In [24]:
print("Número total de registros in df:", df_union.count())

Número total de registros in df: 535803


In [23]:
print('Distinct count in DataFrame df is :', df_union.distinct().count())


Distinct count in DataFrame df is : 535803


In [39]:
from pyspark.sql.functions import countDistinct

In [ ]:
#df_union = df_union.select(countDistinct("icao_empresa_aérea", "icao_aeródromo_destino", "icao_aeródromo_origem"))

In [40]:
df_union.show(10)

+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|código_autorização|código_justificativa|código_tipoLinha|icao_aeródromo_destino|icao_aeródromo_origem|icao_empresa_aérea|número_voo|   partida_prevista|       partida_real|situação_voo|
+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-01-19 14:45:00|2021-01-19 19:22:00|                 7|                 N/A|               C|                  SBEG|                 SBBE|               MWM|      9702|2021-01-19 12:35:00|2021-01-19 17:17:00|   REALIZADO|
|2021-01-26 12:05:00|2021-01-26 12:04:00|                 7|                 N/A|           

In [42]:
df_union.count()


535803

In [43]:
df_union.columns

['chegada_prevista',
 'chegada_real',
 'código_autorização',
 'código_justificativa',
 'código_tipoLinha',
 'icao_aeródromo_destino',
 'icao_aeródromo_origem',
 'icao_empresa_aérea',
 'número_voo',
 'partida_prevista',
 'partida_real',
 'situação_voo']

In [46]:
df_union.distinct().count()

535803

In [47]:
row = df_union.count()

In [48]:
df_union

DataFrame[chegada_prevista: string, chegada_real: string, código_autorização: string, código_justificativa: string, código_tipoLinha: string, icao_aeródromo_destino: string, icao_aeródromo_origem: string, icao_empresa_aérea: string, número_voo: string, partida_prevista: string, partida_real: string, situação_voo: string]

In [50]:
row = df_union.count()
col = len(df_union.columns)
print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')



Dimension of the Dataframe is: (535803, 12)
Number of Rows are: 535803
Number of Columns are: 12


In [58]:
df_union.select("icao_empresa_aérea").show()

+------------------+
|icao_empresa_aérea|
+------------------+
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
|               MWM|
+------------------+
only showing top 20 rows



In [64]:
df_union.groupBy("icao_empresa_aérea").count().show(50)

+------------------+------+
|icao_empresa_aérea| count|
+------------------+------+
|               CLX|   715|
|               ETH|  1255|
|               ACA|   329|
|               KAL|   956|
|               SOO|   160|
|               TTL|  2149|
|               SID|  7867|
|               IBE|   656|
|               BAW|   273|
|               SKU|   425|
|               MWM|  1258|
|               AZN|   150|
|               LPE|    62|
|               TAP|  3452|
|               GLO|115496|
|               THY|  1398|
|               DLH|   707|
|               SAS|    19|
|               KLM|  1195|
|               UAL|  1934|
|               LAN|  1186|
|               SLM|    24|
|               AFR|  1091|
|               MPH|  1143|
|               DAL|   765|
|               PTB|  9694|
|               TIW|    39|
|               GTI|  1208|
|               LTG|  3618|
|               TPA|   771|
|               AAL|  2023|
|               FDX|   717|
|               PAM|

In [60]:
df_union.orderBy("icao_empresa_aérea").show()

+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|código_autorização|código_justificativa|código_tipoLinha|icao_aeródromo_destino|icao_aeródromo_origem|icao_empresa_aérea|número_voo|   partida_prevista|       partida_real|situação_voo|
+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-01-05 08:20:00|2021-01-05 07:51:00|                 0|                 N/A|               X|                  KMIA|                 SBGL|               AAL|      0904|2021-01-04 23:45:00|2021-01-04 23:38:00|   REALIZADO|
|2021-01-25 08:20:00|2021-01-25 07:53:00|                 0|                 N/A|           

df_union.filter(onibusDF.col("icao_empresa_aérea").equalTo("715M-10")).show()

'icao_aeródromo_destino',
 'icao_aeródromo_origem',
df_union.groupBy("icao_empresa_aérea").count().show(50)


In [65]:
df_union.groupBy("icao_aeródromo_destino").count().show(50)

+----------------------+-----+
|icao_aeródromo_destino|count|
+----------------------+-----+
|                  SBPS| 5310|
|                  KATL|  337|
|                  LFPG|  612|
|                  SBIH|  301|
|                  SBSJ|   12|
|                  OTHH|  776|
|                  SCCF|    4|
|                  SBCH| 1562|
|                  SBCR|  176|
|                  PANC|  132|
|                  SBSG| 6055|
|                  SBSO|   95|
|                  SBFN| 1148|
|                  SBLO| 2185|
|                  LTBA|   94|
|                  SBBV| 1139|
|                  GVAC|  336|
|                  SNGA|   30|
|                  SBRP| 3075|
|                  SBJP| 3858|
|                  OMDB|  372|
|                  SWLB|  101|
|                  SBMO| 6629|
|                  SBSV|19801|
|                  SNTF|  253|
|                  SBEG|10994|
|                  SWKO|  269|
|                  SNOX|   38|
|                  FQMA|    1|
|       

In [83]:
#SBSV
#TAM
#df_union.filter(df_union.col("icao_aeródromo_destino").equalTo("TAM")).show()

df_union.select('icao_empresa_aérea').where(df_union.icao_empresa_aérea == 'TAM').show()

+------------------+
|icao_empresa_aérea|
+------------------+
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
|               TAM|
+------------------+
only showing top 20 rows



In [ ]:
#df_union.select(['situação_voo','icao_empresa_aérea']).where(df_union.('situação_voo'=='REALIZADO',).show(1)

In [89]:
df_union.select(['icao_empresa_aérea', 'situação_voo']).where(df_union.situação_voo == 'REALIZADO').show()

+------------------+------------+
|icao_empresa_aérea|situação_voo|
+------------------+------------+
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
|               MWM|   REALIZADO|
+------------------+------------+
only showing top 20 rows



In [90]:
df_union.columns

['chegada_prevista',
 'chegada_real',
 'código_autorização',
 'código_justificativa',
 'código_tipoLinha',
 'icao_aeródromo_destino',
 'icao_aeródromo_origem',
 'icao_empresa_aérea',
 'número_voo',
 'partida_prevista',
 'partida_real',
 'situação_voo']

In [92]:
# & (df_union.situação_voo==df.número_voo))
df_union.filter((df_union.icao_empresa_aérea=='TAM') & (df_union.icao_aeródromo_origem!=df_union.icao_aeródromo_destino).show()

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_17432/3881447961.py, line 2)

In [96]:
df_union [df_union.situação_voo.isin("REALIZADO", "TAM")].show()

+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|código_autorização|código_justificativa|código_tipoLinha|icao_aeródromo_destino|icao_aeródromo_origem|icao_empresa_aérea|número_voo|   partida_prevista|       partida_real|situação_voo|
+-------------------+-------------------+------------------+--------------------+----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-01-19 14:45:00|2021-01-19 19:22:00|                 7|                 N/A|               C|                  SBEG|                 SBBE|               MWM|      9702|2021-01-19 12:35:00|2021-01-19 17:17:00|   REALIZADO|
|2021-01-26 12:05:00|2021-01-26 12:04:00|                 7|                 N/A|           